In [ ]:
%%capture
!rm -r sample_data/
!pip install transformers datasets -U sentence-transformers bertopic

In [ ]:
from datasets import load_dataset, Dataset
from sentence_transformers import util
from bertopic import BERTopic
import umap
import hdbscan
import numpy as np
import pandas as pd
import joblib
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
dataset = load_dataset("rjac/all-the-news-2-1-Component-one-embedding")

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Using custom data configuration rjac--all-the-news-2-1-Component-one-embedding-f2c65fc4e73cbdb8


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/rjac___parquet/rjac--all-the-news-2-1-Component-one-embedding-f2c65fc4e73cbdb8/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

The dataset has been preprocess and extract the ingotmation of the text in a Embedding representation using sentence_transformers

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article_idx', 'year', 'month', 'paragraph_position', 'paragraph', 'embedding'],
        num_rows: 2475520
    })
})

As part of the first Iteration , The process will generate a new cluster using a Dense Algorith using a Community Detection Algo in wich take into consideration the Cosine Similarity to cluster + the min size of the Cluster (Comunity)

In [ ]:
dataset.set_format(type='tf', columns=['embedding'],output_all_columns=True)

Dimensional reduction using UMAP

In [ ]:
N=100_000

In [ ]:
subset = dataset["train"].shuffle(7643).select(range(N))
corpus_embeddings = subset["embedding"]

In [ ]:
%%time
umap_model = umap.UMAP(n_neighbors=250, n_components=6, metric='cosine')
umap_corpus_embeddings = umap_model.fit_transform(corpus_embeddings)
joblib.dump(umap_model, 'umap_model.pkl')

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


CPU times: user 37min 4s, sys: 4min, total: 41min 5s
Wall time: 8min 1s


Dimmention reduction using Autoencoders

In [ ]:
N=100_000

In [ ]:
subset = dataset["train"].shuffle(17643).select(range(N))
corpus_embeddings = subset["embedding"]

In [ ]:
import tensorflow as tf

class DenseTranspose(tf.keras.layers.Layer):
    def __init__(self, dense, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.dense = dense
        self.activation = tf.keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.biases = self.add_weight(name="bias",
                                      shape=self.dense.input_shape[-1],
                                      initializer="zeros")
        super().build(batch_input_shape)

    def call(self, inputs):
        Z = tf.matmul(inputs, self.dense.weights[0], transpose_b=True)
        return self.activation(Z + self.biases)



hidden1 = tf.keras.layers.Dense(450,activation="tanh")
hidden2 = tf.keras.layers.Dense(180,activation="tanh")
encoder_output = tf.keras.layers.Dense(6,activation="tanh")
decoder_output = tf.keras.layers.Dense(384,activation="tanh")

drop1 = tf.keras.layers.Dropout(0.3)
drop2 = tf.keras.layers.Dropout(0.4)

encoder = tf.keras.Sequential([
    drop1
    ,hidden1
    ,hidden2
    ,drop2
    ,encoder_output
])

decoder = tf.keras.Sequential([
    DenseTranspose(encoder_output,activation="tanh")
    ,DenseTranspose(hidden2,activation="tanh")
    ,DenseTranspose(hidden1,activation="tanh")
])

autoencoder = tf.keras.Sequential([encoder, decoder])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.65)
autoencoder.compile(loss="mse", optimizer=optimizer)

In [ ]:
%%time
autoencoder.fit(corpus_embeddings,corpus_embeddings, epochs=10)

Epoch 1/10
3125/3125 [==============================] - 11s 2ms/step - loss: 0.0024
Epoch 2/10
3125/3125 [==============================] - 7s 2ms/step - loss: 0.0022
Epoch 3/10
3125/3125 [==============================] - 7s 2ms/step - loss: 0.0021
Epoch 4/10
3125/3125 [==============================] - 7s 2ms/step - loss: 0.0021
Epoch 5/10
3125/3125 [==============================] - 7s 2ms/step - loss: 0.0020
Epoch 6/10
3125/3125 [==============================] - 7s 2ms/step - loss: 0.0020
Epoch 7/10
3125/3125 [==============================] - 7s 2ms/step - loss: 0.0020
Epoch 8/10
3125/3125 [==============================] - 7s 2ms/step - loss: 0.0020
Epoch 9/10
3125/3125 [==============================] - 7s 2ms/step - loss: 0.0020
Epoch 10/10
3125/3125 [==============================] - 7s 2ms/step - loss: 0.0020
CPU times: user 1min 36s, sys: 16.2 s, total: 1min 52s
Wall time: 1min 15s


Auto encoder all the data with 10 Epochs

In [ ]:
subset = dataset["train"].shuffle(7643).train_test_split(train_size=0.8)

In [ ]:
train_embeddings = subset["train"]["embedding"]
test_embeddings = subset["test"]["embedding"]

In [ ]:
import tensorflow as tf

class DenseTranspose(tf.keras.layers.Layer):
    def __init__(self, dense, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.dense = dense
        self.activation = tf.keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.biases = self.add_weight(name="bias",
                                      shape=self.dense.input_shape[-1],
                                      initializer="zeros")
        super().build(batch_input_shape)

    def call(self, inputs):
        Z = tf.matmul(inputs, self.dense.weights[0], transpose_b=True)
        return self.activation(Z + self.biases)

#hidden1 = tf.keras.layers.Dense(264,activation="relu")
hidden2 = tf.keras.layers.Dense(600,activation="relu")
encoder_output = tf.keras.layers.Dense(32,activation="sigmoid")

#decoder_output = tf.keras.layers.Dense(384,activation="tanh")

drop1 = tf.keras.layers.Dropout(0.3)
drop2 = tf.keras.layers.Dropout(0.2)
drop3 = tf.keras.layers.Dropout(0.1)

encoder = tf.keras.Sequential([
    drop1
 #   ,hidden1
 #   ,drop2
    ,hidden2
 #   ,drop3
    ,encoder_output
])

decoder = tf.keras.Sequential([
    DenseTranspose(encoder_output,activation="relu")
   ,DenseTranspose(hidden2,activation="linear")
 #   ,DenseTranspose(hidden1,activation="tanh")
])

autoencoder = tf.keras.Sequential([encoder, decoder])

optim = tf.keras.optimizers.Adam(
    learning_rate=0.05,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1,
    amsgrad=False,
    name='Adam'
)

autoencoder.compile(loss=tf.keras.losses.CosineSimilarity(axis=1), optimizer=optim)

In [ ]:
%%time
autoencoder.fit(train_embeddings,train_embeddings,validation_data=(test_embeddings,test_embeddings), epochs=4)

Epoch 1/4
61888/61888 [==============================] - 159s 3ms/step - loss: -0.6579 - val_loss: -0.7080
Epoch 2/4
61888/61888 [==============================] - 158s 3ms/step - loss: -0.6999 - val_loss: -0.7317
Epoch 3/4
61888/61888 [==============================] - 157s 3ms/step - loss: -0.7186 - val_loss: -0.7450
Epoch 4/4
61888/61888 [==============================] - 158s 3ms/step - loss: -0.7294 - val_loss: -0.7528
CPU times: user 14min 32s, sys: 2min 21s, total: 16min 54s
Wall time: 10min 31s


In [ ]:
!rm -r /content/encoder_model

rm: cannot remove '/content/encoder_model': No such file or directory


In [ ]:
encoder.compile(loss=tf.keras.losses.CosineSimilarity(axis=1), optimizer=optim)

In [ ]:
encoder.save('encoder_model')

INFO:tensorflow:Assets written to: encoder_model/assets


INFO:tensorflow:Assets written to: encoder_model/assets


BertTopic

In [ ]:
# Class init with a tensorflow model
import tensorflow as tf
class DimensionalityReduction:
    def __init__(self,model_path,norm=False):
        self.model_path = model_path
        self.model = tf.keras.models.load_model(model_path)
        self.norm = norm

    def fit(self, X):
        return self

    def transform(self, X):
        X_transform = self.model.predict(X)
        if self.norm:
            X_transform = np.array([v/np.linalg.norm(v) for v in X_transform])
        return X_transform

In [ ]:
dim_model = DimensionalityReduction("/content/encoder_model")

In [ ]:
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words="english", min_df=1,max_df=0.95)

In [ ]:
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=15,min_samples=5,metric='euclidean',cluster_selection_method='leaf')

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(
    verbose=True
    ,calculate_probabilities=False
    ,vectorizer_model=vectorizer_model
    ,diversity=0.4
    ,umap_model=dim_model
    ,hdbscan_model=hdbscan_model
)

In [ ]:
N=10_000

In [ ]:
subset = dataset["train"].shuffle(27643).select(range(N))

In [ ]:
subset.set_format(type='numpy', columns=['embedding'],output_all_columns=True)

In [ ]:
corpus_embeddings = subset["embedding"]
paragraph = subset["paragraph"]

In [ ]:
topics_class,probs = topic_model.fit_transform(paragraph,corpus_embeddings)

2022-07-21 22:59:05,756 - BERTopic - The dimensionality reduction algorithm did not contain the `y` parameter and therefore the `y` parameter was not used
2022-07-21 22:59:06,161 - BERTopic - Reduced dimensionality
2022-07-21 22:59:12,090 - BERTopic - Clustered reduced embeddings


In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,9015,-1_said_people_like_trump
1,0,108,0_trump_house_impeachment_president
2,1,87,1_million_earnings share_share_earnings
3,2,64,2_points_scored_game_rebounds
4,3,58,3_galaxy_phone_screen_pro
5,4,58,4_percent_rate_fed_sp
6,5,56,5_brexit_eu_britain_party
7,6,55,6_innings_yankees_runs_inning
8,7,43,7_iran_iraq_syria_said
9,8,42,8_health_care_obamacare_house


In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,0,9522,0_democrats_administration_clinton_lawmakers
1,1,478,1_rhp_mcilroy_3pointer_oilers


In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,6808,-1_people_trump_mr_company
1,0,2931,0_trump_president_percent_house
2,1,64,1_game_scored_rebounds_lead
3,2,55,2_innings_yankees_runs_inning
4,3,43,3_iran_iraq_syria_islamic
5,4,40,4_match_williams_round_win
6,5,23,5_coronavirus_virus_cases_outbreak
7,6,18,6_immigration_children_trump_border
8,7,18,7_goal_scored_season_game


In [ ]:
topic_model.visualize_barchart(top_n_topics=12)

In [ ]:
topic_model.visualize_barchart(top_n_topics=12)

Initial cluster for 500k Embeddings

In [ ]:
N=500_000

In [ ]:
subset = dataset["train"].shuffle(7643).select(range(N))

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/rjac___parquet/rjac--all-the-news-2-1-Component-one-embedding-f2c65fc4e73cbdb8/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8/cache-e0f5ce1b33e4325b.arrow


In [ ]:
corpus_embeddings = subset["embedding"]
paragraph = subset["paragraph"]

In [ ]:
corpus_embeddings_encoded = encoder.predict(corpus_embeddings)

In [ ]:
corpus_embeddings_norm = np.array([v/np.linalg.norm(v) for v in corpus_embeddings_encoded])

In [ ]:
#%%time
#umap_model = umap.UMAP(n_neighbors=350, n_components=5, metric='cosine')
#umap_corpus_embeddings = umap_model.fit_transform(corpus_embeddings)
#joblib.dump(umap_model, 'umap_model.pkl')

In [ ]:
%%time
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=300,min_samples=100,metric='euclidean', cluster_selection_epsilon = 0.15,cluster_selection_method='leaf',prediction_data=True)
clusters = hdbscan_model.fit_predict(corpus_embeddings_norm)

CPU times: user 27min 26s, sys: 8.02 s, total: 27min 34s
Wall time: 45min 11s


In [ ]:
print(len(clusters))
pd.DataFrame({"cluster":clusters}).value_counts().reset_index().sort_values("cluster")

800000


,cluster,0
0,-1,656268
10,0,25
11,1,23
8,2,273
5,3,2915
7,4,383
3,5,14365
9,6,63
6,7,509
4,8,9067


We will need to remove duplicates given that some cluster show same sentences and then try will lower threshold

In [ ]:
from collections import Counter 
c = Counter([w for p,c in zip(paragraph,clusters) for w in p.split() if c == 3])

In [ ]:
c.most_common(60)

[('the', 77110),
 ('to', 40954),
 ('a', 40252),
 ('and', 36363),
 ('of', 32257),
 ('in', 31097),
 ('was', 24412),
 ('that', 16943),
 ('on', 14210),
 ('he', 12646),
 ('for', 11695),
 ('his', 11559),
 ('with', 11220),
 ('The', 10203),
 ('her', 10044),
 ('said', 9401),
 ('at', 9320),
 ('is', 8238),
 ('had', 8023),
 ('police', 7498),
 ('who', 7161),
 ('not', 6998),
 ('she', 6660),
 ('from', 6535),
 ('were', 6516),
 ('as', 6482),
 ('by', 6473),
 ('an', 6257),
 ('said.', 6216),
 ('after', 5385),
 ('have', 5292),
 ('been', 4993),
 ('told', 4742),
 ('be', 4408),
 ('they', 4275),
 ('has', 4248),
 ('Mr.', 3931),
 ('this', 3842),
 ('it', 3793),
 ('according', 3737),
 ('are', 3586),
 ('their', 3567),
 ('but', 3517),
 ('about', 3381),
 ('when', 3330),
 ('He', 3291),
 ('I', 3253),
 ('him', 3201),
 ('two', 3107),
 ('Police', 2920),
 ('found', 2845),
 ('one', 2819),
 ('up', 2808),
 ('or', 2731),
 ('A', 2708),
 ('—', 2606),
 ('officers', 2582),
 ('also', 2529),
 ('County', 2509),
 ('shot', 2420)]

In [ ]:
#dataset.push_to_hub("rjac/all-the-news-2-1-Component-one-embedding",max_shard_size="2GB",private=False,token="")

In [ ]:
dataset.set_format(type='pt', columns=['embedding'],output_all_columns=True)

In [ ]:
subset = dataset.shuffle(5843).select(range(1_000_000))

In [ ]:
corpus_embeddings = subset["embedding"]
paragraph = subset["paragraph"]

In [ ]:
import time
import torch

In [ ]:
print("Start clustering")
start_time = time.time()
clusters_ = util.community_detection(torch.from_numpy(corpus_embeddings_norm), min_community_size=50, threshold=0.7,batch_size=2048)
print("Clustering done after {:.2f} sec".format(time.time() - start_time))

Start clustering


KeyboardInterrupt: ignored

In [ ]:
len(clusters_)

In [ ]:
total_clustered = 0
for i, cluster in enumerate(clusters):
    total_clustered += len(cluster)
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:1]:
        print("\t", paragraph[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-1:]:
        print("\t", paragraph[sentence_id])


Cluster 1, #157 Elements 
	 LONDON (Reuters) - Two of Iran’s biggest OPEC rivals, Saudi Arabia and Iraq, are bagging its European oil market share by cranking up production of their lookalike crude grades before the restart of U.S. sanctions designed to halt exports from the Islamic Republic.  Iran faces curbs on its energy sector from early November after U.S. President Donald Trump walked away in May from a 2015 deal aimed at restraining Tehran’s nuclear program.  Iran’s crude is mainly bought by refiners in China, India, Japan and South Korea, although it has customers in Turkey and the European Union. Buyers seeking an alternative cannot simply opt for any crude on the market.   “As oil sanctions against Iran take effect, perhaps in combination with production problems elsewhere, maintaining global supply might be very challenging and would come at the expense of maintaining an adequate spare capacity cushion,” the International Energy Agency warned recently. [IEA/M]  “Thus, the m

In [ ]:
total_clustered/100_000

0.04501

In [ ]:
subset.select(clusters[0])[0:10]["paragraph"]

["LONDON (Reuters) - Two of Iran’s biggest OPEC rivals, Saudi Arabia and Iraq, are bagging its European oil market share by cranking up production of their lookalike crude grades before the restart of U.S. sanctions designed to halt exports from the Islamic Republic.  Iran faces curbs on its energy sector from early November after U.S. President Donald Trump walked away in May from a 2015 deal aimed at restraining Tehran’s nuclear program.  Iran’s crude is mainly bought by refiners in China, India, Japan and South Korea, although it has customers in Turkey and the European Union. Buyers seeking an alternative cannot simply opt for any crude on the market.   “As oil sanctions against Iran take effect, perhaps in combination with production problems elsewhere, maintaining global supply might be very challenging and would come at the expense of maintaining an adequate spare capacity cushion,” the International Energy Agency warned recently. [IEA/M]  “Thus, the market outlook could be far 